<a href="https://colab.research.google.com/github/zia207/Survival_Analysis_R/blob/main/Colab_Notebook/02_07_07_06_survival_analysis_deep_survival_gpu_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![All-test](http://drive.google.com/uc?export=view&id=1bLQ3nhDbZrCCqy_WCxxckOne2lgVvn3l)

# 2.7.6 Deep Survival Model - GUP


DeepSurv (Katzman et al., 2018) introduced a breakthrough by replacing the linear predictor of the Cox model with a deep neural network while retaining the same partial likelihood objective. This elegant extension preserves the interpretability of hazard ratios (when needed) and the ability to handle right-censored data, but dramatically increases modeling flexibility.

This tutorial demonstrates DeepSurv in R using torch with automatic CUDA (GPU) acceleration.


## Overview


**DeepSurv** is a deep learning extension of the Cox proportional hazards model. Introduced by Katzman et al. (2018), it replaces the linear predictor in the Cox model with a **fully connected neural network**, enabling the model to capture **nonlinear relationships** and **complex interactions** among covariates while preserving the interpretability of survival risk.

Unlike traditional machine learning models that predict point estimates, DeepSurv outputs a **risk score** that is used within the **partial likelihood framework** of Cox regression. This makes it particularly suitable for:

- High-dimensional clinical or omics data  
- Electronic health records with complex feature interactions  
- Scenarios where proportional hazards hold approximately, but linearity does not  

This tutorial demonstrates how to implement DeepSurv in **R using the `torch` package**, with and without hyperparameter tuning, using a simulated melanoma dataset.


###  How DeepSurv Works



The Cox model specifies the hazard for individual $i$ at time $t$ as:

$$
h_i(t) = h_0(t) \exp(\mathbf{x}_i^\top \boldsymbol{\beta})
$$

where:

- $h_0(t)$ is the baseline hazard (nonparametric)  
- $\mathbf{x}_i$ is the vector of covariates  
- $\boldsymbol{\beta}$ are coefficients  

The **partial likelihood** avoids estimating $h_0(t)$ and focuses on ranking events.

DeepSurv -  Replacing Linearity with a Neural Network

DeepSurv replaces $\mathbf{x}_i^\top \boldsymbol{\beta}$ with a **neural network risk function** $f_\theta(\mathbf{x}_i)$:

$$
h_i(t) = h_0(t) \exp(f_\theta(\mathbf{x}_i))
$$

The **negative log partial likelihood** is used as the loss:

$$
\mathcal{L}(\theta) = -\sum_{i: \delta_i = 1} \left[ f_\theta(\mathbf{x}_i) - \log \left( \sum_{j \in \mathcal{R}(t_i)} \exp(f_\theta(\mathbf{x}_j)) \right) \right]
$$

where:

- $\delta_i = 1$ if event occurred (uncensored)
- $\mathcal{R}(t_i)$ is the risk set at time $t_i$ (all subjects with $t_j \geq t_i$)

In practice, we sort by descending time and compute cumulative sums for efficiency—exactly as implemented in the `cox_nll()` function below.




### Why DeepSurv is better than classic Cox in many cases


| Advantage                              | Real-world example                              |
|----------------------------------------|--------------------------------------------------|
| Captures non-linear effects            | Tumor thickness > 4 mm is much worse than linear assumption |
| Learns interactions automatically     | Ulceration + thickness together is far worse than either alone |
| Scales to thousands of features        | Works with genomics, radiomics, EHR data        |
| Easy to add images, text, time-series  | Multi-modal deep survival models (DeepSurv + CNNs, etc.) |



## Setup R in Python Runtype - Install {rpy2}
{rpy2} is a Python package that provides an interface to the R programming language, allowing Python users to run R code, call R functions, and manipulate R objects directly from Python. It enables seamless integration between Python and R, leveraging R's statistical and graphical capabilities while using Python's flexibility. The package supports passing data between the two languages and is widely used for statistical analysis, data visualization, and machine learning tasks that benefit from R's specialized libraries.

In [ ]:
!pip uninstall rpy2 -y
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

Found existing installation: rpy2 3.5.17
Uninstalling rpy2-3.5.17:
  Successfully uninstalled rpy2-3.5.17
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp312-cp312-linux_x86_64.whl size=316569 sha256=89ae16fad1fffae78000c08d45854029920c240c8c9745051d438ffc816d52e2
  Stored in directory: /root/.cache/pip/wheels/00/26/d5/d5e8c0b039915e785be870270e4a9263e5058168a03513d8cc
Successfully built rpy2


## Mount Google Drive

In [ ]:
## Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## DeepSurv in R


This is a complete working example of DeepSurv on the Melanoma dataset from the MASS package in R, using the torch package for deep learning.

Key features of this implementation:

- A flexible multi-layer perceptron with ReLU activations and dropout
- Exact implementation of the Cox partial negative log-likelihood using pure torch operations
- Mini-batch training with Adam optimizer and proper handling of the risk set
- Robust indexing to avoid common R/torch pitfalls (e.g., “argument not interpretable as logical”, S3 dispatch errors)
- Automatic tracking of training and validation loss
- Final evaluation via Harrell’s C-index and visualization of predicted risk stratification



### Install Torch


To run this code, you need to have the `torch` package installed. You can install it from CRAN and then install the appropriate LibTorch backend (CPU or CUDA) by running:


In [ ]:
%%R
install.packages("torch")
torch::install_torch()   # will download the right LibTorch (CPU or CUDA)


You can verify that torch is installed correctly by running:


In [ ]:
%%R
library(torch)
x <- array(runif(8), dim = c(2, 2, 2))
y <- torch_tensor(x, dtype = torch_float64())
y
identical(x, as_array(y))

### Install Required R Packages


Following R packages are required to run this notebook. If any of these packages are not installed, you can install them using the code below:


In [ ]:
%%R
packages <-c(
		 'tidyverse',
		 'tidyr',
		 'Hmisc',
	   'survival',
		 'survMisc',
		 'survminer',
		 'MASS',
		 'torch'

		 )


### Install missing packages

In [ ]:
%%R
# Install missing packages
new.packages <- packages[!(packages %in% installed.packages(lib='drive/My Drive/R/')[,"Package"])]
if(length(new.packages)) install.packages(new.packages, lib='drive/My Drive/R/')
devtools::install_github("ItziarI/WeDiBaDis", lib='drive/My Drive/R/')


### Verify Installation

In [ ]:
%%R
.libPaths('drive/My Drive/R')
# Verify installation
cat("Installed packages:\n")
print(sapply(packages, requireNamespace, quietly = TRUE))

Installed packages:
  tidyverse survivalsvm    survival    survcomp    survMisc   survminer 
       TRUE        TRUE        TRUE        TRUE        TRUE        TRUE 
       MASS 
       TRUE 


### Load Packages

In [ ]:
%%R
.libPaths('drive/My Drive/R')
# Verify installation
cat("Installed packages:\n")
print(sapply(packages, requireNamespace, quietly = TRUE))

Installed packages:
  tidyverse survivalsvm    survival    survcomp    survMisc   survminer 
       TRUE        TRUE        TRUE        TRUE        TRUE        TRUE 
       MASS 
       TRUE 


In [ ]:
%%R
# Check loaded packages
cat("Successfully loaded packages:\n")
print(search()[grepl("package:", search())])

###  Simulated Melanoma Dataset


We simulate a melanoma dataset (`n = 2000`) with known nonlinear effects (e.g., interaction between tumor thickness and ulceration, sinusoidal terms). The data includes:

- `time`: observed survival time  
- `event`: binary event indicator (1 = death, 0 = censored)  
- Covariates: `age`, `sex`, `thickness`, `ulcer`, `year`

This simulation ensures ground-truth performance is measurable (expected C-index ≈ 0.84).



In [ ]:
%%R
sim_melanoma <- function(n = 2000) {
  sex       <- rbinom(n, 1, 0.6)
  age       <- rnorm(n, 52, 16) %>% pmax(15) %>% pmin(90)
  thickness <- rlnorm(n, 0.5, 1.1)
  ulcer     <- rbinom(n, 1, 0.4)
  year      <- round(runif(n, 1962, 1977))

  lp <- (0.02 * scale(age)[,1] -
         0.45 * sex +
         0.35 * log1p(thickness) +
         0.90 * ulcer -
         0.07 * scale(year)[,1] +
         0.3 * sin(scale(thickness)[,1] * 2) +
         0.5 * ulcer * log1p(thickness))

  shape <- 1.3; scale <- 8.0
  U <- runif(n)
  T_true <- scale * (-log(U) / exp(lp))^(1/shape)
  C <- rexp(n, rate = 0.07)
  time  <- pmin(T_true, C)
  event <- as.numeric(T_true <= C)

  data.frame(time, event,
             sex = factor(sex, labels = c("Male","Female")),
             age, thickness, ulcer = factor(ulcer, labels = c("No","Yes")), year)
}

df <- sim_melanoma(2000)
cat("Simulated n =", nrow(df), "| Events =", sum(df$event), "\n")

## 3. Data Preprocessing

In [ ]:
%%R
train_idx <- sample(seq_len(nrow(df)), 1400)
val_idx   <- sample(setdiff(seq_len(nrow(df)), train_idx), 300)
test_idx  <- setdiff(seq_len(nrow(df)), c(train_idx, val_idx))

train_df <- df[train_idx, ]; val_df <- df[val_idx, ]; test_df <- df[test_idx, ]

num_cols <- c("age", "thickness", "year")
means <- colMeans(train_df[num_cols])
sds   <- apply(train_df[num_cols], 2, sd)

scale_df <- function(d) {
  d[num_cols] <- scale(d[num_cols], center = means, scale = sds)
  d %>% mutate(
    sex   = as.numeric(sex)   - 1,
    ulcer = as.numeric(ulcer) - 1
  )
}

train_df <- scale_df(train_df)
val_df   <- scale_df(val_df)
test_df  <- scale_df(test_df)

### Convert to **GPU-enabled tensors**

In [ ]:
%%R
# # Detect device (GPU if available, otherwise CPU)
device <- if (torch::cuda_is_available()) {
  cat("GPU detected — using CUDA!\n")
  torch_device("cuda")
} else {
  cat("No GPU — using CPU\n")
  torch_device("cpu")
}

# covert to tensor
to_tensor <- function(x) torch_tensor(x, dtype = torch_float(), device = device)

x_train <- to_tensor(as.matrix(train_df[, c("age","thickness","year","sex","ulcer")]))
x_val   <- to_tensor(as.matrix(val_df[,   c("age","thickness","year","sex","ulcer")]))
x_test  <- to_tensor(as.matrix(test_df[,  c("age","thickness","year","sex","ulcer")]))

y_time_train  <- to_tensor(train_df$time)
y_event_train <- to_tensor(train_df$event)
y_time_val    <- to_tensor(val_df$time)
y_event_val   <- to_tensor(val_df$event)

## DeepSurv Model and Loss Function (GPU-Compatible)

In [ ]:
%%R
make_deepsurv_model <- function(input_dim = 5, hidden1 = 128, hidden2 = 64, hidden3 = 32,
                                dropout1 = 0.3, dropout2 = 0.2) {
  model <- nn_module(
    "DeepSurv",
    initialize = function(input_dim) {
      self$net <- nn_sequential(
        nn_linear(input_dim, hidden1), nn_relu(), nn_dropout(dropout1),
        nn_linear(hidden1, hidden2),   nn_relu(), nn_dropout(dropout2),
        nn_linear(hidden2, hidden3),   nn_relu(),
        nn_linear(hidden3, 1)
      )
    },
    forward = function(x) self$net(x)$squeeze(-1)
  )(input_dim)

  # Move model to device (CPU or CUDA)
  model$to(device = device)
}

cox_nll <- function(risk, time, event) {
  ord <- torch_argsort(time, descending = TRUE)
  risk <- risk[ord]
  event <- event[ord]$bool()
  if (event$sum()$item() == 0) return(risk$mean() * 0)

  risk <- risk - torch_mean(risk)
  risk <- torch_clamp(risk, min = -10, max = 10)

  hazard <- torch_exp(-risk)
  cum_hazard <- torch_cumsum(hazard, dim = 1L)
  cum_hazard <- torch_clamp(cum_hazard, min = 1e-8)
  log_cum_hazard <- torch_log(cum_hazard)

  uncensored <- torch_nonzero(event)$squeeze()
  if (uncensored$dim() == 0) uncensored <- uncensored$unsqueeze(0)

  loss <- -(risk[uncensored] - log_cum_hazard[uncensored])$mean()
  loss
}

### Case Study 1: Fixed Hyperparameters (with CUDA)

In [ ]:
%%R
model <- make_deepsurv_model()
optimizer <- optim_adam(model$parameters, lr = 5e-4, weight_decay = 1e-4)

epochs <- 500; batch_size <- 128
train_losses <- numeric(epochs); val_losses <- numeric(epochs)

for (epoch in 1:epochs) {
  model$train()
  perm <- torch_randperm(x_train$size(1), device = device) + 1L
  i <- 1L; batch_loss <- 0; nbat <- 0
  while (i <= x_train$size(1)) {
    end <- min(i + batch_size - 1, x_train$size(1))
    idx <- perm[i:end]

    xb <- x_train$index_select(1, idx)
    tb <- y_time_train$index_select(1, idx)
    eb <- y_event_train$index_select(1, idx)

    optimizer$zero_grad()
    risk <- model(xb)
    loss <- cox_nll(risk, tb, eb)
    loss$backward()
    optimizer$step()

    batch_loss <- batch_loss + loss$item(); nbat <- nbat + 1
    i <- i + batch_size
  }
  train_losses[epoch] <- batch_loss / nbat

  if (epoch %% 50 == 0 || epoch == epochs) {
    model$eval()
    val_loss <- with_no_grad({ cox_nll(model(x_val), y_time_val, y_event_val)$item() })
    val_losses[epoch] <- val_loss
    cat(sprintf("Epoch %3d | Train: %.5f | Val: %.5f\n", epoch, train_losses[epoch], val_loss))
    model$train()
  } else {
    val_losses[epoch] <- NA
  }
}

#### Evaluate and visualize results

In [ ]:
%%R
model$eval()
test_risk_fixed <- as.numeric(with_no_grad({ model(x_test)$cpu() }))
cindex_fixed <- Hmisc::rcorr.cens(-test_risk_fixed, Surv(test_df$time, test_df$event))[["C Index"]]
cat("\n C-index (Fixed HP):", round(cindex_fixed, 4), "\n")

#### Loss Curve

In [ ]:
%%R
loss_df_fixed <- data.frame(epoch = 1:epochs,
                            Training = train_losses,
                            Validation = val_losses) %>%
  pivot_longer(-epoch, names_to = "Type", values_to = "Loss")

p_loss_fixed <- ggplot(loss_df_fixed, aes(x = epoch, y = Loss, color = Type)) +
  geom_line(size = 1.1) +
  geom_point(data = subset(loss_df_fixed, Type == "Validation" & !is.na(Loss)), size = 3) +
  scale_color_manual(values = c("Training" = "#2E86AB", "Validation" = "#A23B72")) +
  labs(title = "DeepSurv (Fixed HP) — Loss Curve",
       subtitle = paste("Test C-index =", round(cindex_fixed, 4)),
       x = "Epoch", y = "Cox Negative Log-Likelihood") +
  theme_minimal(base_size = 13) + theme(legend.position = "top")
p_loss_fixed

#### Kaplan–Meier Plot

In [ ]:
%%R
# --- KM Plot ---
test_df_plot_fixed <- test_df
test_df_plot_fixed$risk <- test_risk_fixed
test_df_plot_fixed$risk_group <- ifelse(test_risk_fixed >= median(test_risk_fixed), "High risk", "Low risk")
fit_km_fixed <- survfit(Surv(time, event) ~ risk_group, data = test_df_plot_fixed)

p_km_fixed <- ggsurvplot(fit_km_fixed, data = test_df_plot_fixed,
                         risk.table = TRUE, pval = TRUE,
                         palette = c("#E41A1C", "#377EB8"),
                         legend.labs = c("High risk", "Low risk"),
                         title = "DeepSurv Risk Stratification (Fixed HP)")$plot

# Display plots
p_km_fixed

### DeepSurv With Hyperparameter Tuning

We perform **random search** over:
- Learning rate: $10^{-5}$ to $10^{-2.5}$
- Weight decay: $10^{-6}$ to $10^{-2}$
- Architecture sizes and dropout rates

Each trial trains for 200 epochs; the best model is retrained for 500 epochs.

> ️ **Note**: Each trial model is moved to `device`. Final evaluation moves output to CPU.

In [ ]:
%%R
best_val_loss <- Inf
best_config <- list()

for (trial in 1:10) {
  lr <- 10^runif(1, -5, -2.5)
  wd <- 10^runif(1, -6, -2)
  d1 <- runif(1, 0.1, 0.5)
  d2 <- runif(1, 0.1, 0.3)
  h1 <- sample(c(64,128,256),1); h2 <- sample(c(32,64,128),1); h3 <- sample(c(16,32,64),1)

  model_t <- make_deepsurv_model(5, h1, h2, h3, d1, d2)
  opt_t <- optim_adam(model_t$parameters, lr = lr, weight_decay = wd)

  for (ep in 1:150) {
    model_t$train()
    perm <- torch_randperm(x_train$size(1), device = device) + 1L
    i <- 1L
    while (i <= x_train$size(1)) {
      end <- min(i + 128 - 1, x_train$size(1))
      idx <- perm[i:end]
      xb <- x_train$index_select(1, idx)
      tb <- y_time_train$index_select(1, idx)
      eb <- y_event_train$index_select(1, idx)
      opt_t$zero_grad()
      risk <- model_t(xb)
      loss <- cox_nll(risk, tb, eb)
      loss$backward()
      opt_t$step()
      i <- i + 128
    }
  }

  model_t$eval()
  vloss <- with_no_grad({ cox_nll(model_t(x_val), y_time_val, y_event_val)$item() })

  if (vloss < best_val_loss) {
    best_val_loss <- vloss
    best_config <- list(lr=lr, wd=wd, h1=h1, h2=h2, h3=h3, d1=d1, d2=d2, state=model_t$state_dict())
  }
}

# Retrain best model
model_tuned <- make_deepsurv_model(5, best_config$h1, best_config$h2, best_config$h3,
                                   best_config$d1, best_config$d2)
model_tuned$load_state_dict(best_config$state)
optimizer_tuned <- optim_adam(model_tuned$parameters, lr = best_config$lr, weight_decay = best_config$wd)

# Full training
for (epoch in 1:500) {
  model_tuned$train()
  perm <- torch_randperm(x_train$size(1), device = device) + 1L
  i <- 1L
  while (i <= x_train$size(1)) {
    end <- min(i + 128 - 1, x_train$size(1))
    idx <- perm[i:end]
    xb <- x_train$index_select(1, idx)
    tb <- y_time_train$index_select(1, idx)
    eb <- y_event_train$index_select(1, idx)
    optimizer_tuned$zero_grad()
    risk <- model_tuned(xb)
    loss <- cox_nll(risk, tb, eb)
    loss$backward()
    optimizer_tuned$step()
    i <- i + 128
  }
}

#### Evaluate and visualize results

In [ ]:
%%R
model_tuned$eval()
test_risk_tuned <- as.numeric(with_no_grad({ model_tuned(x_test)$cpu() }))
cindex_tuned <- Hmisc::rcorr.cens(-test_risk_tuned, Surv(test_df$time, test_df$event))[["C Index"]]
cat("\n C-index (Tuned HP):", round(cindex_tuned, 4), "\n")

#### Loss Curve

In [ ]:
%%R
# --- Loss Plot ---
loss_df_tuned <- data.frame(epoch = 1:500,
                            Training = train_losses_tuned,
                            Validation = val_losses_tuned) %>%
  pivot_longer(-epoch, names_to = "Type", values_to = "Loss")

p_loss_tuned <- ggplot(loss_df_tuned, aes(x = epoch, y = Loss, color = Type)) +
  geom_line(size = 1.1) +
  geom_point(data = subset(loss_df_tuned, Type == "Validation" & !is.na(Loss)), size = 3) +
  scale_color_manual(values = c("Training" = "#2E86AB", "Validation" = "#A23B72")) +
  labs(title = "DeepSurv (Tuned HP) — Loss Curve",
       subtitle = paste("Test C-index =", round(cindex_tuned, 4)),
       x = "Epoch", y = "Cox Negative Log-Likelihood") +
  theme_minimal(base_size = 13) + theme(legend.position = "top")
p_loss_tuned

#### Kaplan–Meier Plot

In [ ]:
%%R
# --- KM Plot ---
test_df_plot_tuned <- test_df
test_df_plot_tuned$risk <- test_risk_tuned
test_df_plot_tuned$risk_group <- ifelse(test_risk_tuned >= median(test_risk_tuned), "High risk", "Low risk")
fit_km_tuned <- survfit(Surv(time, event) ~ risk_group, data = test_df_plot_tuned)

p_km_tuned <- ggsurvplot(fit_km_tuned, data = test_df_plot_tuned,
                         risk.table = TRUE, pval = TRUE,
                         palette = c("#E41A1C", "#377EB8"),
                         legend.labs = c("High risk", "Low risk"),
                         title = "DeepSurv Risk Stratification (Tuned HP)")$plot

# Display plots
p_km_tuned


## Resources

- **Original Paper**: Katzman et al. (2018). [DeepSurv](https://doi.org/10.1186/s12874-018-0482-1)  
- **R `torch`**: https://torch.mlverse.org/  
- **Survival Analysis in R**: *Therneau & Grambsch (2000). Modeling Survival Data*  
- **Code Repository**: [github.com/jaredleekatzman/DeepSurv](https://github.com/jaredleekatzman/DeepSurv) (Python)  
- **Alternative R Packages**: `survival`, `rms`, `mlr3proba`, `torchopt`